In [7]:
# BASIC IMPORTS
import sys
sys.path.append(r'..\src')

import pandas as pd

import pylab as plt

import seaborn as sns

import numpy as np

import re

from src import getGraphic
from ufc_fights import set_names_upper, set_name_winner, fix_columns, set_nan_columns, add_fighterid
from ufc_fighters import load_ufc_fighters, load_ufc_fighters_styles, set_fighter_upper, clean_fighters

In [8]:
# MACHINE LEARNING IMPORTS
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor as RFR
rfr=RFR()

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

import h2o
from h2o.automl import H2OAutoML

In [116]:
fighter = pd.read_csv(r'..\data\raw_fighter_details.csv')
data = pd.read_csv(r'..\data\data.csv')
ufc_fighters = pd.read_csv(r'..\data\ufc_fighters_official.csv')
ufc_styles = pd.read_csv(r'..\data\ufc_fighters_styles.csv')
ufc_odds = pd.read_csv('../data/ufc_odds.csv')

In [117]:
# Las reglas oficiales de la UFC no entraron hasta el año 2001, así que solo cogemos combates por encima del año 2002
data = data[data.date>'2002-01-01']
# B_draw y R_draw son columnas con todos los valores 0
data = data.drop(['B_draw','R_draw'], axis=1)
# Eliminamos DRAW en Winner para ir probando
data = data[data.Winner!='Draw']
# Eliminamos la columna LOCATION
data = data.drop('location', axis=1)

In [118]:
set_names_upper(data)
fix_columns(data)
set_nan_columns(data)
data = data.drop('fight_id', axis=1)

In [119]:
set_names_upper(ufc_odds)
ufc_odds

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,THIAGO SANTOS,JOHNNY WALKER,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,ALEX OLIVEIRA,NIKO PRICE,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,MISHA CIRKUNOV,KRZYSZTOF JOTKO,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,ALEXANDER HERNANDEZ,MIKE BREEDEN,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0
4,JOE SOLECKI,JARED GORDON,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,DUANE LUDWIG,DARREN ELKINS,-155.0,135,64.516129,135.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Blue,...,NaN,1.0,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN
4892,JOHN HOWARD,DANIEL ROBERTS,-210.0,175,47.619048,175.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,Punch,1.0,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN
4893,BRENDAN SCHAUB,CHASE GORMLEY,-260.0,220,38.461538,220.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,Punches,1.0,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN
4894,MIKE PIERCE,JULIO PAULINO,-420.0,335,23.809524,335.000000,3/21/2010,"Broomfield, Colorado, USA",USA,Red,...,NaN,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
x = data.set_index(['R_fighter','B_fighter']).join(ufc_odds[['R_fighter','B_fighter','R_odds','B_odds']].set_index(['R_fighter','B_fighter']),
                                               rsuffix='_dcha', how='left')

In [122]:
x.dropna(axis=0, inplace=True)
x = x.reset_index(col_level=0)
x = x.reset_index(col_level=0)
x = x.drop('index', axis=1)

In [137]:
x.sort_values(by=['date'], axis=0, ascending=False)
x.reset_index(drop=True, inplace=True)

In [138]:
x

,R_fighter,B_fighter,date,Winner,title_bout,weight_class,B_avg_KD,B_avg_SIG_STR_pct,B_avg_TD_pct,B_avg_SUB_ATT,...,B_PCT_LEG,R_PCT_LEG,B_PCT_DISTANCE,R_PCT_DISTANCE,B_PCT_CLINCH,R_PCT_CLINCH,B_PCT_GROUND,R_PCT_GROUND,R_odds,B_odds
0,AALON CRUZ,SPIKE CARLYLE,2020-02-29,Blue,False,Featherweight,0.017578,0.449412,0.25,0.143188,...,83.0,83.0,37.0,37.0,68.0,68.0,70.0,70.0,-190.0,165.0
1,AARON PHILLIPS,MATT HOBAR,2014-08-23,Blue,False,Bantamweight,0.000000,0.250000,0.00,0.000000,...,100.0,33.0,20.0,35.0,100.0,50.0,70.0,75.0,145.0,-155.0
2,AARON RILEY,JOE BRAMMER,2010-05-29,Red,False,Lightweight,0.000000,0.150000,0.00,0.000000,...,100.0,58.0,16.0,22.0,68.0,55.0,70.0,75.0,-200.0,170.0
3,AARON RILEY,JUSTIN SALAS,2013-07-27,Blue,False,Lightweight,0.000000,0.535000,0.13,0.000000,...,78.0,72.0,38.0,26.0,77.0,69.0,77.0,75.0,220.0,-240.0
4,AARON ROSA,MATT LUCAS,2011-11-12,Red,False,LightHeavyweight,0.017578,0.449412,0.25,0.143188,...,83.0,80.0,37.0,39.0,68.0,58.0,70.0,70.0,-120.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4489,ZHANG LIPENG,BRENDAN O'REILLY,2014-08-23,Red,False,Lightweight,0.017578,0.449412,0.25,0.143188,...,83.0,86.0,37.0,36.0,68.0,100.0,70.0,50.0,140.0,-150.0
4490,ZHANG LIPENG,CHRIS WADE,2015-01-18,Blue,False,Lightweight,0.000000,0.000000,1.00,1.000000,...,83.0,92.0,0.0,35.0,68.0,88.0,70.0,62.0,505.0,-605.0
4491,ZHANG LIPENG,KAJAN JOHNSON,2015-05-16,Blue,False,Lightweight,0.000000,0.310000,0.44,0.000000,...,100.0,89.0,30.0,37.0,100.0,78.0,33.0,70.0,-115.0,-105.0
4492,ZUBAIRA TUKHUGOV,ERNEST CHAVEZ,2014-10-04,Red,False,Featherweight,0.000000,0.415000,0.50,0.000000,...,77.0,100.0,38.0,40.0,71.0,31.0,50.0,22.0,-260.0,240.0


In [141]:
# FUNCTION FOR SELECT THE LAST OR FIRST FIGHT OF A CONCRETE FIGHTER

#  i = index of the fighter's fight, 0 means the last fight, -1 means first fight
def select_fight_row(df, name, i): 
    
    df_temp = df[(df['R_fighter'] == name) | (df['B_fighter'] == name)]  # filter df on fighter's name
    
    df_temp.reset_index(drop=True, inplace=True) #  as we created a new temporary dataframe, we have to reset indexes
    
    idx = max(df_temp.index)  #  get the index of the oldest fight
    
    if i > idx:  #  if we are looking for a fight that didn't exist, we return nothing
        
        return
    
    arr = df_temp.iloc[i].values
    
    return arr

select_fight_row(x, 'GLOVER TEIXEIRA', 0) #  we get the last fight of Jon Jones, for example

array(['ALEXANDER GUSTAFSSON', 'GLOVER TEIXEIRA',
       Timestamp('2017-05-28 00:00:00'), 'Red', False, 'LightHeavyweight',
       0.13037109375, 0.41404296875, 0.52060546875, 0.5341796875,
       0.015625, 1.90869140625, 1.66845703125, 337.69580078125,
       542.68798828125, 26, 1, 9, 3, 0, 0, 2, 3, 3, 1, 'Orthodox', 187.96,
       193.04, 205.0, 0.326904296875, 0.4843896484375, 0.486435546875,
       0.00341796875, 0.0, 4.82421875, 2.447265625, 353.4814453125,
       935.56005859375, 30, 2, 9, 4, 0, 0, 3, 4, 2, 0, 'Orthodox', 195.58,
       200.66, 205.0, 37.0, 30.0, 62.0, 60.0, 42.0, 43.0, 77.0, 63.0,
       100.0, 81.0, 40.0, 39.0, 74.0, 57.0, 58.0, 81.0, -300.0, 250.0],
      dtype=object)

In [142]:
# GET ALL ACTIVE FIGHTERS ACCORDING TO THE LIMIT DATE

def list_fighters(df, limit_date):
    
    df_temp = df[df['date'] > limit_date]
    
    set_R = set(df_temp['R_fighter'])
    set_B = set(df_temp['B_fighter'])
    
    fighters = list(set_R.union(set_B))
    
    return fighters

fighters = list_fighters(x, '2002-01-01')
len(fighters)

1608

In [143]:
# FUNCTION FOR BUILD A DATAFRAME WHICH HAS LAST/FIRST/... FIGHTS OF SELECTED FIGHTERS

def build_df(df, fighters, i):
    
    arr = [select_fight_row(df, fighters[f], i) for f in range(len(fighters)) if select_fight_row(df, fighters[f], i) is not None]
    
    cols = [col for col in df] 
    
    df_fights = pd.DataFrame(data=arr, columns=cols)
    
    df_fights.drop_duplicates(inplace=True)
    
    df_fights['title_bout'] = df_fights['title_bout'].replace({True: 1, False: 0})
    
    df_fights.drop(['R_fighter', 'B_fighter', 'date'], axis=1, inplace=True)
    
    return df_fights

df_train = build_df(x, fighters, 0)
df_test = build_df(x, fighters, 1)

In [144]:
df_train.shape, df_test.shape

((1424, 67), (1274, 67))

In [145]:
# AMOUNT OF WINNERS RED OR BLUE IN TRAIN AND TEST DATAFRAMES

print(len(df_train[df_train['Winner'] == 'Blue']))
print(len(df_train[df_train['Winner'] == 'Red']))

print(len(df_test[df_test['Winner'] == 'Blue']))
print(len(df_test[df_test['Winner'] == 'Red']))

573
851
525
749
